In [1]:
import pandas as pd

In [3]:
# import data
df1_1 = pd.read_csv('证券2.csv')
df1_2 = pd.read_csv('证券1.csv')
df1_3 = pd.concat([df1_1, df1_2], ignore_index=True)
df1 = df1_3[df1_3['民营指数'] >= 1]


In [4]:
df2 = pd.read_csv('TRD_Mnth.csv')

df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])
df2['统计日期'] = pd.to_datetime(df2['交易月份'])

/tmp/ipykernel_90/1366149548.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])


In [5]:



df1_sorted = df1.sort_values(by=['统计日期', '其他持股比例'], ascending=[True, False])

grouped_returns = pd.DataFrame()

periods = {
    9: (1, 4),
    3: (5, 8),
    6: (9, 12)
}


In [6]:

for month, (start_month, end_month) in periods.items():
    # 筛选出分组月份的数据
    df1_monthly = df1_sorted[df1_sorted['统计日期'].dt.month == month]

    # 对每一年的数据进行操作
    for year in df1_monthly['统计日期'].dt.year.unique():
        # 获取当前年份和上一年的数据
        current_year_data = df1_monthly[df1_monthly['统计日期'].dt.year == year]
        previous_year_data = df1_monthly[df1_monthly['统计日期'].dt.year == year - 1]

        # 对当前年份的数据进行分组
        current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')


        # 遍历每个分组
        for group in range(5):
            # 获取上一年同一分组的证券代码
            previous_group_codes = previous_year_data[previous_year_data['证券代码'].isin(current_year_data[current_year_data['分组'] == group]['证券代码'])]['证券代码']

            # 计算加权回报率
            for month in range(start_month, end_month + 1):
                # 获取对应月份的数据
                df2_monthly = df2[(df2['统计日期'].dt.year == year) & (df2['统计日期'].dt.month == month)]

                # 筛选出属于当前分组的股票
                df2_group = df2_monthly[df2_monthly['证券代码'].isin(previous_group_codes)]

                print("******************************")
                print(current_year_data['其他持股比例'])
                print(df2_monthly['证券代码'])
                print(current_year_data['分组'])
                print("******************************")
                
                # 计算加权回报率
                total_market_value = df2_group['个股市值'].sum()
                df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
                weighted_return = df2_group['加权回报率'].sum()

                # 将结果存储到grouped_returns中
                new_data = pd.DataFrame({
                    '年份': [year],
                    '月份': [month],
                    '分组': [group],
                    '加权回报率': [weighted_return]
                })

                grouped_returns = pd.concat([grouped_returns, new_data], ignore_index=True)


/tmp/ipykernel_90/1249222003.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')


******************************
20860     94.910
118428    92.990
20144     92.420
118463    91.890
20427     90.710
           ...  
19809      7.300
118323     4.623
20136        NaN
20658        NaN
118459       NaN
Name: 其他持股比例, Length: 1535, dtype: float64
34             1
205            2
369            4
522            5
689            6
           ...  
572365    900952
572536    900953
572707    900955
572853    900956
572982    900957
Name: 证券代码, Length: 2524, dtype: int64
20860     4.0
118428    4.0
20144     4.0
118463    4.0
20427     4.0
         ... 
19809     0.0
118323    0.0
20136     NaN
20658     NaN
118459    NaN
Name: 分组, Length: 1535, dtype: float64
******************************
******************************
20860     94.910
118428    92.990
20144     92.420
118463    91.890
20427     90.710
           ...  
19809      7.300
118323     4.623
20136        NaN
20658        NaN
118459       NaN
Name: 其他持股比例, Length: 1535, dtype: float64
35             1
206        

/tmp/ipykernel_90/1249222003.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')
/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']


******************************
27085     103.600
122226     92.410
27033      91.860
122262     91.700
26945      90.970
           ...   
26027       8.870
122117      5.868
25960         NaN
26361         NaN
121690        NaN
Name: 其他持股比例, Length: 1626, dtype: float64
46             1
217            2
381            4
534            5
701            6
           ...  
572377    900952
572548    900953
572719    900955
572865    900956
572994    900957
Name: 证券代码, Length: 2546, dtype: int64
27085     4.0
122226    4.0
27033     4.0
122262    4.0
26945     4.0
         ... 
26027     0.0
122117    0.0
25960     NaN
26361     NaN
121690    NaN
Name: 分组, Length: 1626, dtype: float64
******************************
******************************
27085     103.600
122226     92.410
27033      91.860
122262     91.700
26945      90.970
           ...   
26027       8.870
122117      5.868
25960         NaN
26361         NaN
121690        NaN
Name: 其他持股比例, Length: 1626, dtype: float64
47    

/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']


...

/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']


******************************
182308    100.04
182282    100.00
182342    100.00
182405    100.00
182406    100.00
           ...  
96590       6.66
95091        NaN
183167       NaN
183313       NaN
183422       NaN
Name: 其他持股比例, Length: 4011, dtype: float64
162            1
327            2
490            4
646            5
812            6
           ...  
571917    900947
572088    900948
572493    900952
572664    900953
573110    900957
Name: 证券代码, Length: 5371, dtype: int64
182308    4.0
182282    4.0
182342    4.0
182405    4.0
182406    4.0
         ... 
96590     0.0
95091     NaN
183167    NaN
183313    NaN
183422    NaN
Name: 分组, Length: 4011, dtype: float64
******************************
******************************
182308    100.04
182282    100.00
182342    100.00
182405    100.00
182406    100.00
           ...  
96590       6.66
95091        NaN
183167       NaN
183313       NaN
183422       NaN
Name: 其他持股比例, Length: 4011, dtype: float64
163            1
328        

/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']


******************************
182308    100.04
182282    100.00
182342    100.00
182405    100.00
182406    100.00
           ...  
96590       6.66
95091        NaN
183167       NaN
183313       NaN
183422       NaN
Name: 其他持股比例, Length: 4011, dtype: float64
164            1
329            2
492            4
648            5
814            6
           ...  
571919    900947
572090    900948
572495    900952
572666    900953
573112    900957
Name: 证券代码, Length: 5399, dtype: int64
182308    4.0
182282    4.0
182342    4.0
182405    4.0
182406    4.0
         ... 
96590     0.0
95091     NaN
183167    NaN
183313    NaN
183422    NaN
Name: 分组, Length: 4011, dtype: float64
******************************
******************************
182308    100.04
182282    100.00
182342    100.00
182405    100.00
182406    100.00
           ...  
96590       6.66
95091        NaN
183167       NaN
183313       NaN
183422       NaN
Name: 其他持股比例, Length: 4011, dtype: float64
165            1
330        

/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
/tmp/ipykernel_90/1249222003.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']


In [ ]:
print(grouped_returns.head())


In [ ]:
grouped_returns.to_excel('计算.xlsx', index=False)